<a href="https://colab.research.google.com/github/anujaprasads/anujashajan/blob/main/inceptionv3withcrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow as tf

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
# Keras
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename

In [ ]:
num=14
for i in range(2,num):
    if num%i==0:
        print("not prime")
        break
else:
    print("prime")        


not prime


In [ ]:
list=eval(input("enter a list:"))
list.sort()
print(list)
list.sort(reverse=True)
print(list)

enter a list:[20,9,3,1,2,70]
[1, 2, 3, 9, 20, 70]
[70, 20, 9, 3, 2, 1]


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/movingvehicletrain/croptraindataaugmentation'
valid_path = '/content/drive/MyDrive/croptrain/croppedvalidation'

In [ ]:

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/movingvehicletrain/croptraindataaugmentation/*')
print(len(folders))

5


In [ ]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/movingvehicletrain/croptraindataaugmentation',
                                                 target_size = (224, 224),
                                                 batch_size = 1,
                                                 class_mode = 'categorical')

Found 2525 images belonging to 5 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/croptrain/croppedvalidation',
                                            target_size = (224, 224),
                                            batch_size = 1,
                                            class_mode = 'categorical')

Found 789 images belonging to 5 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=70,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/70
2525/2525 [==============================] - 638s 251ms/step - loss: 6.5344 - accuracy: 0.8028 - val_loss: 59.5065 - val_accuracy: 0.3105
Epoch 2/70
2525/2525 [==============================] - 472s 187ms/step - loss: 3.6815 - accuracy: 0.9208 - val_loss: 130.9787 - val_accuracy: 0.1331
Epoch 3/70
2525/2525 [==============================] - 475s 188ms/step - loss: 3.5886 - accuracy: 0.9307 - val_loss: 79.9171 - val_accuracy: 0.3878
Epoch 4/70
2525/2525 [==============================] - 467s 185ms/step - loss: 2.6393 - accuracy: 0.9473 - val_loss: 80.4515 - val_accuracy: 0.4411
Epoch 5/70
2525/2525 [==============================] - 465s 184ms/step - loss: 2.6463 - accuracy: 0.9564 - val_loss: 92.9175 - val_accuracy: 0.5146
Epoch 6/70
2525/2525 [==============================] - 471s 186ms/step - loss: 2.4778 - accuracy: 0.9600 - val_loss: 116.3583 - val_accuracy: 0.3257
Epoch 7/70
2525/2525 [==============================] - 471s 187ms/step - loss: 1.8483 - accuracy: 0.966

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: ignored

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/movingvehicletrain/modeldataaugmentationcropcontour_inceptionwithnv.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model=load_model('/content/drive/MyDrive/movingvehicletrain/modeldataaugmentation_inceptionwithnv.h5')

In [ ]:

import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

TypeError: ignored

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

array([[0.000000e+00, 1.160042e-06, 0.000000e+00, 4.228590e-11,
        9.999988e-01],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        1.000000e+00],
       [1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       ...,
       [7.251335e-23, 0.000000e+00, 0.000000e+00, 1.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        1.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        1.000000e+00]], dtype=float32)

In [ ]:

import numpy as np
y_pred = np.argmax(y_pred, axis=1)
y_pred


array([4, 4, 0, 4, 4, 0, 4, 3, 1, 3, 0, 4, 3, 0, 1, 4, 4, 4, 4, 1, 4, 4,
       1, 0, 4, 0, 4, 4, 1, 4, 0, 0, 4, 4, 4, 3, 3, 1, 1, 0, 3, 3, 3, 0,
       4, 4, 4, 0, 1, 3, 3, 3, 4, 3, 4, 1, 4, 3, 1, 3, 4, 0, 1, 1, 4, 4,
       4, 4, 0, 4, 0, 4, 4, 3, 1, 4, 3, 3, 0, 4, 1, 0, 4, 4, 4, 1, 0, 3,
       4, 2, 4, 1, 1, 1, 4, 1, 1, 1, 0, 4, 4, 0, 3, 3, 0, 3, 4, 4, 1, 3,
       0, 4, 4, 4, 0, 4, 3, 0, 3, 4, 4, 3, 4, 4, 1, 4, 0, 1, 0, 0, 0, 0,
       4, 0, 4, 0, 0, 4, 4, 3, 0, 4, 0, 1, 0, 4, 4, 1, 4, 0, 4, 2, 4, 0,
       0, 1, 3, 4, 4, 0, 3, 1, 0, 1, 1, 0, 4, 2, 1, 3, 0, 4, 0, 4, 2, 0,
       0, 1, 1, 1, 2, 0, 0, 2, 4, 3, 1, 4, 1, 1, 1, 1, 3, 1, 0, 0, 4, 3,
       0, 0, 1, 4, 4, 3, 4, 4, 3, 4, 1, 3, 4, 0, 0, 4, 4, 4, 0, 1, 1, 3,
       1, 1, 3, 2, 1, 0, 0, 3, 4, 3, 3, 4, 0, 3, 1, 3, 4, 0, 4, 0, 3, 4,
       0, 3, 0, 0, 0, 3, 0, 0, 1, 4, 1, 1, 4, 0, 4, 0, 1, 0, 0, 4, 4, 0,
       0, 3, 0, 4, 0, 0, 3, 0, 3, 4, 4, 1, 4, 0, 0, 0, 4, 3, 0, 2, 0, 3,
       4, 1, 0, 4, 3, 4, 2, 0, 0, 3, 0, 3, 4, 1, 3,

In [ ]:
img=image.load_img('/content/drive/MyDrive/croptrain/test_yolo/3w/0.jpg',target_size=(224,224))
x=image.img_to_array(img)
x=x/255
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
y_pred = model.predict(img_data)

In [ ]:
y_pred

array([[0.15840551, 0.27382395, 0.15188923, 0.22291647, 0.09556445,
        0.09740039]], dtype=float32)

In [ ]:
a=np.argmax(y_pred, axis=1)
print(a)

[1]


In [ ]:
import cv2
from skimage import transform
import matplotlib.pyplot as plt
import skimage.io
bgImageFile = "/content/drive/MyDrive/movingvehicletrain/train/2w/ 0.jpg" 
bg = cv2.imread(bgImageFile)
for i in range(0,269):
    bgImageFile = "/content/drive/MyDrive/movingvehicletrain/train/2w/% d.jpg" % i
    print ("Opening background", bgImageFile)
    bg = cv2.imread(bgImageFile)       
    transformed= transform.rotate(bg,angle=-15,cval=255)
    rgbImg = cv2.cvtColor(transformed, cv2.COLOR_BGR2RGB)
    filename_surf1 = '/content/drive/MyDrive/movingvehicletrain/train/2wrotate/frame'+str(i)+'.jpg'
    


Opening background /content/drive/MyDrive/movingvehicletrain/train/2w/ 0.jpg


AttributeError: ignored

In [ ]:
from PIL import Image

for i in range(173,286):
    bgImageFile = "/content/drive/MyDrive/croptrain/croppedtrain/2w/%d.jpg" % i
    #print ("Opening background", bgImageFile)
    #print(bgImageFile)
    #bg = cv2.imread(bgImageFile)
    img = Image.open(bgImageFile)
    rotated = img.rotate(10)
    rotated.save('/content/drive/MyDrive/croptrain/croppedtrain/2wrotate/frame'+str(i)+'.jpg')

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
 
# Function to rename multiple files
def main():
    i=0
    folder = "/content/drive/MyDrive/movingvehicletrain/train_yolo"
    #folder1 = "/content/drive/MyDrive/movingvehicletrain/train/2w"
    for count, filename in enumerate(os.listdir(folder)):
        
        dst = f"{str(i)}.jpg"
        src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
        dst =f"{folder}/{dst}"
        i+=1 
        # rename() function will
        # rename all the files
        os.rename(src, dst)
 
# Driver Code
if __name__ == '__main__':
     
    # Calling main() function
    main()

In [ ]:
from skimage import transform
from PIL import Image
import cv2
import matplotlib.pyplot as plt

image ="/content/drive/MyDrive/movingvehicletrain/train/2w/ 0.jpg"
img=cv2.imread(image)
transform=transform.flip(img,angle=-20,cval=255)
plt.imshow(img)
plt.show()

#plt.imshow(transform)
plt.show()

AttributeError: ignored

In [ ]:
from PIL import Image
from skimage import transform
import cv2
import matplotlib.pyplot as plt
for i in range(0,10):
    bgImageFile = "/content/drive/MyDrive/movingvehicletrain/train_yolo/%d.jpg" % i
    basewidth = 300
    img1=cv2.imread(bgImageFile)
#transform=transform.flip(img,angle=-10,cval=255)
    
    #img = Image.open(bgImageFile)
    #wpercent = (basewidth / float(img.size[0]))
    #hsize = int((float(img.size[1]) * float(wpercent)))
    #img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    #plt.imshow(img1)
    #plt.show()   
    rotated = img1.rotate(15)
    rotated.save('/content/drive/MyDrive/movingvehicletrain/roation15/frame'+str(i)+'.jpg')

AttributeError: ignored

In [ ]:
from PIL import Image

for i in range(0,116):
    bgImageFile = "/content/drive/MyDrive/movingvehicletrain/train_yolo/%d.jpg" % i
    #print ("Opening background", bgImageFile)
    #print(bgImageFile)
    #bg = cv2.imread(bgImageFile)
    img = Image.open(bgImageFile)
    rotated = img.rotate(15)
    rotated.save('/content/drive/MyDrive/movingvehicletrain/train_data_Yolo_augmentation_rotate15/frame'+str(i)+'.jpg')

In [ ]:
from PIL import Image

for i in range(173,286):
    bgImageFile = "/content/drive/MyDrive/croptrain/croppedtrain/2w/%d.jpg" % i
    #print ("Opening background", bgImageFile)
    #print(bgImageFile)
    #bg = cv2.imread(bgImageFile)
    img = Image.open(bgImageFile)
    rotated = img.rotate(10)
    rotated.save('/content/drive/MyDrive/croptrain/croppedtrain/2wrotate/frame'+str(i)+'.jpg')

FileNotFoundError: ignored

In [ ]:
import glob
import shutil
import os

src_dir = "/content/drive/MyDrive/movingvehicletrain/train_yolo"
dst_dir = "/content/drive/MyDrive/movingvehicletrain/train_yolo"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)

SameFileError: ignored